## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [6]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [7]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [9]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


## Computing the Hypothesis

In [11]:
print('1/(1 + e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1 + e^{-1}) equals:  tensor([0.7311])


In [12]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [13]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


## Computing the Cost Function

In [15]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [17]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [18]:
cost = losses.mean()

In [19]:
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [20]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

## Whole Training Procedure

In [24]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
  # Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = F.binary_cross_entropy(hypothesis, y_train)
  
  # cost로 H(x) 계산
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번 마다 출력
  if epoch % 100 == 0:
    print(f'Epoch {epoch:4d} / {nb_epochs} Cost : {cost.item():.6f}')

Epoch    0 / 1000 Cost : 0.693147
Epoch  100 / 1000 Cost : 0.134722
Epoch  200 / 1000 Cost : 0.080643
Epoch  300 / 1000 Cost : 0.057900
Epoch  400 / 1000 Cost : 0.045300
Epoch  500 / 1000 Cost : 0.037261
Epoch  600 / 1000 Cost : 0.031672
Epoch  700 / 1000 Cost : 0.027556
Epoch  800 / 1000 Cost : 0.024394
Epoch  900 / 1000 Cost : 0.021888
Epoch 1000 / 1000 Cost : 0.019852


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading Real Data

In [26]:
import numpy as np

In [30]:
xy = np.loadtxt('/content/drive/MyDrive/ds_study/data/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [31]:
print(x_train[:5])
print(y_train[:5])

tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


## Training with Real Data using low-level Binary Cross Entropy Loss

In [34]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
  # Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()
  
  # cost로 H(x) 계산
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 0:
    print(f'Epoch {epoch:3d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch   0/100 Cost: 0.693147
Epoch  10/100 Cost: 0.572727
Epoch  20/100 Cost: 0.539493
Epoch  30/100 Cost: 0.519708
Epoch  40/100 Cost: 0.507066
Epoch  50/100 Cost: 0.498539
Epoch  60/100 Cost: 0.492549
Epoch  70/100 Cost: 0.488209
Epoch  80/100 Cost: 0.484985
Epoch  90/100 Cost: 0.482543
Epoch 100/100 Cost: 0.480661


## Training with Real Data using F.binary_cross_entropy

In [35]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
  # Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = F.binary_cross_entropy(hypothesis, y_train)
  
  # cost로 H(x) 계산
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 0:
    print(f'Epoch {epoch:3d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch   0/100 Cost: 0.693147
Epoch  10/100 Cost: 0.572727
Epoch  20/100 Cost: 0.539493
Epoch  30/100 Cost: 0.519708
Epoch  40/100 Cost: 0.507066
Epoch  50/100 Cost: 0.498539
Epoch  60/100 Cost: 0.492549
Epoch  70/100 Cost: 0.488209
Epoch  80/100 Cost: 0.484985
Epoch  90/100 Cost: 0.482543
Epoch 100/100 Cost: 0.480661


## Checking the Accuracy our Model

In [36]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[0.4103],
        [0.9242],
        [0.2300],
        [0.9411],
        [0.1772]], grad_fn=<SliceBackward0>)


In [39]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(hypothesis.shape)
print(prediction.shape)

torch.Size([759, 1])
torch.Size([759, 1])


In [40]:
print(prediction[:5])

tensor([[False],
        [ True],
        [False],
        [ True],
        [False]])


In [41]:
print(y_train[:5])

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [42]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])
print(correct_prediction.shape)

tensor([[True],
        [True],
        [True],
        [True],
        [True]])
torch.Size([759, 1])


In [45]:
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print(f'accuracy: {accuracy * 100:2.2f}%')

accuracy: 76.68%


## Optional: High-level Implementation with nn.Module

In [46]:
class BinaryClassfier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(8, 1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):
    return self.sigmoid(self.linear(x))

In [47]:
model = BinaryClassfier()

In [49]:
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
  hypothesis = model(x_train)

  cost =  F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 0:
    prediction = hypothesis >= torch.FloatTensor([0.5])
    correct_prediction = prediction.float() == y_train
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f} Accuracy: {accuracy*100:2.2f} %')

Epoch    0/100 Cost: 0.481067 Accuracy: 76.81 %
Epoch   10/100 Cost: 0.479596 Accuracy: 76.94 %
Epoch   20/100 Cost: 0.478420 Accuracy: 77.08 %
Epoch   30/100 Cost: 0.477470 Accuracy: 77.08 %
Epoch   40/100 Cost: 0.476694 Accuracy: 76.94 %
Epoch   50/100 Cost: 0.476053 Accuracy: 76.94 %
Epoch   60/100 Cost: 0.475520 Accuracy: 76.94 %
Epoch   70/100 Cost: 0.475071 Accuracy: 76.94 %
Epoch   80/100 Cost: 0.474691 Accuracy: 76.94 %
Epoch   90/100 Cost: 0.474366 Accuracy: 76.94 %
Epoch  100/100 Cost: 0.474087 Accuracy: 76.94 %
